## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,New Norfolk,AU,2021-08-13 18:27:01,-42.78,147.06,45.72,70,60,1.01,broken clouds
1,1,Tura,IN,2021-08-13 18:27:45,25.52,90.22,75.02,98,100,4.79,overcast clouds
2,2,Muli,MV,2021-08-13 18:27:45,2.92,73.57,83.05,70,75,17.47,broken clouds
3,3,Vaini,TO,2021-08-13 18:27:45,-21.20,-175.20,64.56,94,75,4.61,broken clouds
4,4,Marienburg,PL,2021-08-13 18:27:46,54.04,19.03,75.88,55,73,9.06,broken clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Tura,IN,2021-08-13 18:27:45,25.52,90.22,75.02,98,100,4.79,overcast clouds
2,2,Muli,MV,2021-08-13 18:27:45,2.92,73.57,83.05,70,75,17.47,broken clouds
4,4,Marienburg,PL,2021-08-13 18:27:46,54.04,19.03,75.88,55,73,9.06,broken clouds
9,9,Jamestown,US,2021-08-13 18:27:47,42.10,-79.24,83.08,85,75,7.00,light rain
12,12,Ishigaki,JP,2021-08-13 18:27:48,24.34,124.16,82.27,84,87,21.23,overcast clouds
13,13,Kapaa,US,2021-08-13 18:27:48,22.08,-159.32,80.92,76,40,3.00,light rain
15,15,Marsabit,KE,2021-08-13 18:27:48,2.33,37.99,74.57,41,49,18.81,scattered clouds
17,17,Havre-Saint-Pierre,CA,2021-08-13 18:27:33,50.23,-63.60,75.56,78,75,8.05,broken clouds
19,19,Salinas,US,2021-08-13 18:27:49,36.68,-121.66,86.20,65,1,8.05,clear sky
24,24,Wanning,CN,2021-08-13 18:27:51,18.80,110.40,82.06,84,99,11.99,light rain


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                380
City                   380
Country                378
Date                   380
Lat                    380
Lng                    380
Max Temp               380
Humidity               380
Cloudiness             380
Wind Speed             380
Current Description    380
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                378
City                   378
Country                378
Date                   378
Lat                    378
Lng                    378
Max Temp               378
Humidity               378
Cloudiness             378
Wind Speed             378
Current Description    378
dtype: int64

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Tura,IN,75.02,overcast clouds,25.52,90.22,
2,Muli,MV,83.05,broken clouds,2.92,73.57,
4,Marienburg,PL,75.88,broken clouds,54.04,19.03,
9,Jamestown,US,83.08,light rain,42.10,-79.24,
12,Ishigaki,JP,82.27,overcast clouds,24.34,124.16,
13,Kapaa,US,80.92,light rain,22.08,-159.32,
15,Marsabit,KE,74.57,scattered clouds,2.33,37.99,
17,Havre-Saint-Pierre,CA,75.56,broken clouds,50.23,-63.60,
19,Salinas,US,86.20,clear sky,36.68,-121.66,
24,Wanning,CN,82.06,light rain,18.80,110.40,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [23]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Tura,IN,75.02,overcast clouds,25.52,90.22,Hotel RIKMAN Continental
2,Muli,MV,83.05,broken clouds,2.92,73.57,Medhufushi Island Resort
4,Marienburg,PL,75.88,broken clouds,54.04,19.03,Hotel Centrum Malbork
9,Jamestown,US,83.08,light rain,42.10,-79.24,Hampton Inn & Suites Jamestown
12,Ishigaki,JP,82.27,overcast clouds,24.34,124.16,Hotel Peace Land Ishigakijima


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))